# Feature engineering

An agglomeration of several feature engineering notebooks used for the project.

In [176]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import datetime
import numpy as np

## 1. Merge with demographic data from Iowa census

In [177]:
df = pd.read_csv("/data401/Iowa_Liquor_Sales.csv", iterator=True)

In [179]:
names = [str(i) for i in range(195)]

In [180]:
names[1] = "county"
names[3] = "land_area" # in square meters
names[7] = "population"
names[8] = "under_5" # population under 5 years old
names[9] = "5_to_9"
names[10] = "10_to_14"
names[11] = "15_to_19"
names[12] = "20_to_24"
names[13] = "25_to_29"
names[14] = "30_to_34"
names[15] = "35_to_39"
names[16] = "40_to_44"
names[17] = "45_to_49"
names[18] = "50_to_54"
names[19] = "55_to_59"
names[20] = "60_to_64"
names[21] = "65_to_69"
names[22] = "70_to_74"
names[23] = "75_to_79"
names[24] = "80_to_84"
names[25] = "over_84"
names[64] = "median_age"
names[65] = "male_median_age"
names[66] = "female_median_age"
names[73] = "pop_over_21"
names[74] = "male_over_21"
names[75] = "female_over_21"
names[84] = "white" # races (not latinx) should add up to population
names[85] = "black"
names[86] = "american_indian"
names[87] = "asian"
names[95] = "hawaiian_pacific_islander"
names[101] = "other_race"
names[102] = "multiple_races"
names[114] = "latinx"
names[139] = "head_of_household" # population that is a head of their household
names[141] = "has_child" # population that has 1+ children under 18
names[173] = "average_HH_size" # average size of household
names[189] = "owned_housing" # number of housing units that are owned
names[190] = "rented_housing" # number of housing units that are rented

In [181]:
usecols=[]
for name in names:
    if len(name) > 3:
        usecols.append(name)

In [182]:
demo = pd.read_csv("census.csv", names=names, usecols=usecols, skiprows=1)
demo = demo[pd.notnull(demo['county'])]
demo["county"]=demo["county"].apply(lambda x: x[:-7].lower())
demo.to_csv('demographics.csv',encoding = "utf8")

In [189]:
aggdata = pd.read_csv("aggdata.csv")
merged = aggdata.merge(demo, how="left", on="county")

In [175]:
merged.to_csv('data/aggdata02.csv',encoding = "utf8")

## 2. Added DUI data

In [39]:
agg_iowa = pd.read_csv("data/aggdata02.csv", header=1,
                        names=['date','county','num_purchasers','volume_sold_l','total_retail', 'land_area', 'population', 'under_5', '5_to_9', '10_to_14',
                               '15_to_19', '20_to_24', '25_to_29', '30_to_34', '35_to_39', '40_to_44',
                               '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74',
                               '75_to_79', '80_to_84', 'over_84', 'median_age', 'male_median_age',
                               'female_median_age', 'pop_over_21', 'male_over_21', 'female_over_21',
                               'white', 'black', 'american_indian', 'asian',
                               'hawaiian_pacific_islander', 'other_race', 'multiple_races', 'latinx',
                               'head_of_household', 'has_child', 'average_HH_size', 'owned_housing',
                               'rented_housing'])
duis = pd.read_csv('data/dui.csv',header=0,
                  names=['date','county','num_alc_accidents'])

merged = agg_iowa.merge(duis, how='left')

In [41]:
zerod = merged[merged['date'] <= '2018-06-17'].copy()
averaged = merged[merged['date'] > '2018-06-17'].copy()

In [43]:
# fill NaN values with 0's for data that we have (up until 2018-06-17)
zerod['num_alc_accidents'] = zerod['num_alc_accidents'].fillna(0)
avg_duis = zerod.groupby('county')['num_alc_accidents'].mean().round()
averaged['num_alc_accidents'] = averaged.apply(lambda row: avg_duis[row['county']], axis=1)

In [45]:
merged_no_nan = pd.concat([zerod,averaged])

,date,county,num_purchasers,volume_sold_l,total_retail,land_area,population,under_5,5_to_9,10_to_14,...,hawaiian_pacific_islander,other_race,multiple_races,latinx,head_of_household,has_child,average_HH_size,owned_housing,rented_housing,num_alc_accidents
0,2012-01-08,adams,1,112.94,1491.20,1.096701e+09,4029.0,234.0,209.0,242.0,...,0.0,22.0,12.0,30.0,963.0,788.0,2.28,3099.0,816.0,0.0
1,2012-01-08,appanoose,5,966.80,11837.46,1.287981e+09,12887.0,811.0,805.0,767.0,...,1.0,135.0,52.0,106.0,2718.0,2603.0,2.27,9562.0,3199.0,0.0
2,2012-01-08,audubon,1,235.63,2909.50,1.147264e+09,6119.0,335.0,312.0,403.0,...,1.0,37.0,7.0,24.0,1483.0,1254.0,2.29,4906.0,1077.0,0.0
3,2012-01-08,benton,5,962.46,11928.85,1.855117e+09,26076.0,1645.0,1851.0,1882.0,...,2.0,239.0,69.0,190.0,6138.0,6225.0,2.50,21470.0,4293.0,0.0
4,2012-01-08,black hawk,14,5034.50,71661.42,1.465334e+09,131090.0,8458.0,7823.0,7550.0,...,201.0,2985.0,357.0,3585.0,23861.0,25738.0,2.38,88103.0,37020.0,0.0
5,2012-01-08,bremer,7,2013.07,25115.38,1.127880e+09,24276.0,1414.0,1460.0,1539.0,...,2.0,231.0,47.0,161.0,5612.0,5230.0,2.40,18631.0,3927.0,0.0
6,2012-01-08,buchanan,7,1230.10,15403.85,1.478935e+09,20958.0,1566.0,1588.0,1571.0,...,1.0,227.0,71.0,160.0,4685.0,5212.0,2.53,16842.0,3773.0,0.0
7,2012-01-08,buena vista,6,1346.53,17992.31,1.489023e+09,20260.0,1449.0,1345.0,1317.0,...,100.0,359.0,34.0,3778.0,4047.0,4468.0,2.56,13816.0,5430.0,0.0
8,2012-01-08,butler,3,319.30,3635.16,1.502533e+09,14867.0,982.0,950.0,988.0,...,2.0,111.0,34.0,94.0,3689.0,3279.0,2.39,12225.0,2400.0,0.0
9,2012-01-08,calhoun,1,200.50,2458.15,1.476209e+09,9670.0,550.0,583.0,551.0,...,1.0,60.0,14.0,66.0,2312.0,1942.0,2.22,7751.0,1681.0,0.0


In [46]:
merged_no_nan.to_csv('data/aggdata03.csv', encoding = "utf8")

## 3. Computed variables per capita

In [8]:
data = pd.read_csv("data/aggdata03.csv", header=0,
                  names=['date','county','num_purchasers','volume_sold_l','total_retail', 'land_area', 'population', 'under_5', '5_to_9', '10_to_14',
                       '15_to_19', '20_to_24', '25_to_29', '30_to_34', '35_to_39', '40_to_44',
                       '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74',
                       '75_to_79', '80_to_84', 'over_84', 'median_age', 'male_median_age',
                       'female_median_age', 'pop_over_21', 'male_over_21', 'female_over_21',
                       'white', 'black', 'american_indian', 'asian',
                       'hawaiian_pacific_islander', 'other_race', 'multiple_races', 'latinx',
                       'head_of_household', 'has_child', 'average_HH_size', 'owned_housing',
                       'rented_housing', 'num_alc_accidents'])

,date,county,num_purchasers,volume_sold_l,total_retail,land_area,population,under_5,5_to_9,10_to_14,...,hawaiian_pacific_islander,other_race,multiple_races,latinx,head_of_household,has_child,average_HH_size,owned_housing,rented_housing,num_alc_accidents
0,2012-01-08,adams,1,112.94,1491.20,1.096701e+09,4029.0,234.0,209.0,242.0,...,0.0,22.0,12.0,30.0,963.0,788.0,2.28,3099.0,816.0,0.0
1,2012-01-08,appanoose,5,966.80,11837.46,1.287981e+09,12887.0,811.0,805.0,767.0,...,1.0,135.0,52.0,106.0,2718.0,2603.0,2.27,9562.0,3199.0,0.0
2,2012-01-08,audubon,1,235.63,2909.50,1.147264e+09,6119.0,335.0,312.0,403.0,...,1.0,37.0,7.0,24.0,1483.0,1254.0,2.29,4906.0,1077.0,0.0
3,2012-01-08,benton,5,962.46,11928.85,1.855117e+09,26076.0,1645.0,1851.0,1882.0,...,2.0,239.0,69.0,190.0,6138.0,6225.0,2.50,21470.0,4293.0,0.0
4,2012-01-08,black hawk,14,5034.50,71661.42,1.465334e+09,131090.0,8458.0,7823.0,7550.0,...,201.0,2985.0,357.0,3585.0,23861.0,25738.0,2.38,88103.0,37020.0,0.0


In [11]:
data['pop_density'] = data['population']/data['land_area']
data['volume_per_cap'] = data['volume_sold_l']/data['population']

In [12]:
data.to_csv("data/aggdata04.csv")

## 4. Feature engineering with demographic variables

In [53]:
df = pd.read_csv("data/aggdata04.csv")
df.drop([df.columns[0], "female_over_21"], axis=1, inplace=True)

In [55]:
df["over_65"] = df["65_to_69"] + df["70_to_74"] + df["75_to_79"] + df["80_to_84"] + df["over_84"]

In [56]:
df.drop(["65_to_69","70_to_74","75_to_79","80_to_84",'60_to_64',"over_84",'5_to_9', '10_to_14','30_to_34', '35_to_39', '40_to_44', '45_to_49',
       '50_to_54', '55_to_59','black', 'american_indian', 'latinx',
       'asian', 'hawaiian_pacific_islander','other_race','multiple_races'], axis=1, inplace=True)

In [58]:
def make_prop(df, colname):
    newname = colname + "_prop"
    df[newname] = df[colname] / df["population"]
    df.drop(colname, axis=1, inplace=True)

In [59]:
prop_cols = ['under_5', '15_to_19', '20_to_24', '25_to_29','over_65','pop_over_21', 'male_over_21',
             'white','head_of_household', 'has_child']
for col in prop_cols:
    make_prop(df, col)

In [63]:
df.to_csv('data/aggdata05.csv',encoding = "utf8")

## 5. Added holiday data

In [31]:
df = pd.read_csv("data/aggdata05.csv", header=0,
                names = ['date', 'county', 'num_purchasers', 'volume_sold_l', 'total_retail',
                       'land_area', 'population', 'median_age', 'male_median_age',
                       'female_median_age', 'average_HH_size', 'owned_housing',
                       'rented_housing', 'num_alc_accidents', 'pop_density', 'volume_per_cap',
                       'under_5_prop', '15_to_19_prop', '20_to_24_prop', '25_to_29_prop',
                       'over_65_prop', 'pop_over_21_prop', 'male_over_21_prop', 'white_prop',
                       'head_of_household_prop', 'has_child_prop'])
cal = calendar()
holidays = cal.holidays(start=min(df['date']), end=max(df['date']))
df["date"] = pd.to_datetime(df["date"])

In [24]:
def isHoliday(row, holidays):
    span = datetime.timedelta(weeks=2)
    for date in holidays:
        if row['date'] <= date and row['date'] + span >= date:
            return 1
    return 0

In [32]:
df['isHoliday'] = df.apply(isHoliday, args=(holidays,), axis=1)

In [42]:
df.to_csv("data/aggdata06.csv",encoding = "utf8")

## 6. Conversion to numeric and adding season

In [2]:
# Read the data

df = pd.read_csv("data/aggdata06.csv")

df.dropna(inplace=True)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

def convert_date(row, start_date):
    return ((row - start_date) / 7).days

In [3]:
df2 = pd.read_csv('data/aggdata07.csv', header =0,
                 names = ['num_purchasers', 'volume_sold_l', 'total_retail', 'vodka_bottles',
       'tequila_bottles', 'whiskey_bottles', 'rum_bottles', 'gin_bottles',
       'other_bottles', 'avg_bottle_price', 'avg_vendor_size',
       'avg_vendor_vol']).reset_index().drop(['num_purchasers','volume_sold_l','total_retail'], axis=1)
df3 = df.merge(df2, left_on=['date','county'], right_on=['level_0','level_1']).drop(['level_1','level_0'], axis=1).sort_values(by='date')
#df3.to_csv('data/aggdata08.csv', encoding='utf8')

In [4]:
df3['date'] = pd.to_datetime(df3["date"])
start_date = df3['date'].min()
df3['num_weeks'] = df3['date'].apply(convert_date, args=(start_date,))

In [11]:
df2[df2['volume_per_cap'] >= 0.678961]

,volume_per_cap
county,
dickinson,0.678961


In [83]:
df2.describe()

,vodka_bottles,tequila_bottles,whiskey_bottles,rum_bottles,gin_bottles,other_bottles,avg_bottle_price,avg_vendor_size,avg_vendor_vol
count,17430.000000,17430.000000,17430.000000,17430.000000,17430.000000,17430.000000,17430.000000,17430.000000,17430.000000
mean,2480.473379,353.060700,2414.054733,1027.061159,265.090993,1807.153815,1.953057,375.586554,0.001350
std,7138.876310,1159.734456,5636.543765,2432.258640,826.669717,4849.067156,0.682012,226.908278,0.020747
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277189,0.000000,0.000000
25%,319.250000,25.000000,455.000000,170.000000,18.000000,220.000000,1.506227,205.500000,0.000000
50%,641.000000,67.000000,905.500000,337.500000,46.000000,507.000000,1.833553,326.366667,0.000000
75%,1396.000000,172.000000,1858.750000,742.750000,121.000000,1034.000000,2.225082,493.794444,0.000000
max,85900.000000,19571.000000,70268.000000,33849.000000,10734.000000,65038.000000,9.188235,2339.400000,1.000000


In [7]:
df2 = df[['volume_per_cap','county']].groupby('county').mean()
df2[df2['volume_per_cap'] < 0.05]
quartile1 = list(df2[df2['volume_per_cap'] <= 0.117975].index)
quartile2 = list(df2[df2['volume_per_cap'] <= 0.179457].index)
quartile3 = list(df2[df2['volume_per_cap'] <= 0.222417].index)
quartile4 = list(df2[df2['volume_per_cap'] < 0.678961].index)

In [85]:
def cat_county(x):
    if x in quartile1:
        return 'quartile1'
    if x in quartile2:
        return 'quartile2'
    if x in quartile3:
        return 'quartile3'
    if x in quartile4:
        return 'quartile4'
    return 'max'

In [86]:
df3['county_bin'] = df3['county'].apply(cat_county)

In [87]:
def seasons(x):
    if x.month in [12,1,2]:
        return 'Winter'
    if x.month in [3,4,5]:
        return 'Spring'
    if x.month in [6,7,8]:
        return 'Summer'
    return 'Fall'

In [88]:
df3['Seasons'] = df3['date'].apply(seasons)

In [89]:
df3.drop(['date','county'],axis=1, inplace=True)
df3 = pd.get_dummies(df3, drop_first=True)

In [90]:
df3.to_csv("data/aggdata08.csv", encoding='utf8')